In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def compute_SNR(attack_path, n_layer=-2, adv = False):
  # Load the data from npz file
  data = np.load(attack_path)
  X_nat = data["X_test_nat_adv"][0]
  X_adv = data["X_test_nat_adv"][1]

  if adv:
    X_nat, X_adv = X_adv, X_nat

  # Load the model
  model_path = "-".join(attack_path.split("-")[:-1]) + "-model.h5"
  model = tf.keras.models.load_model(model_path)

  # `n_layer` BE CAREFUL with this, it is the layer number, not the index. Check the model summary to get the layer number

  # print("Model summary: ")
  # model.summary()

  # Get output of layer n_layer when the input to model is X_nat
  layer_output = model.layers[n_layer].output
  intermediate = tf.keras.models.Model(inputs=model.input, outputs=layer_output)
  intermediate_output_nat = intermediate.predict(X_nat)
  intermediate_output_adv = intermediate.predict(X_adv)

  # if the dimension of the intermediate output is greater than 1, then we need to flatten it
  if len(intermediate_output_nat.shape) > 2:
    new_shape = intermediate_output_nat.shape[0],np.prod(intermediate_output_nat.shape[1:])
    intermediate_output_nat = intermediate_output_nat.reshape(new_shape)
    new_shape = intermediate_output_adv.shape[0],np.prod(intermediate_output_adv.shape[1:])
    intermediate_output_adv = intermediate_output_adv.reshape(new_shape)
  

  N_features = intermediate_output_nat.shape[1] # Number of features in the intermediate output

  # Remove centroid from intermediate outputs
  centroid_nat = np.mean(intermediate_output_nat, axis=0)
  centroid_adv = np.mean(intermediate_output_adv, axis=0)
  intermediate_output_nat = intermediate_output_nat - centroid_nat
  intermediate_output_adv = intermediate_output_adv - centroid_adv

  # intermediate_outputs are matrices of shape (n_samples, n_features). Get the covariance matrix
  cov_nat = np.cov(intermediate_output_nat, rowvar=False)
  cov_adv = np.cov(intermediate_output_adv, rowvar=False)

  # Get the eigenvalues and eigenvectors of the covariance matrices
  eigval_nat, eigvec_nat = np.linalg.eig(cov_nat)
  eigval_adv, eigvec_adv = np.linalg.eig(cov_adv)

  # Calculate the radius of intermediate_output_nat along the eigenvectors
  Rs_nat = np.zeros((eigvec_nat.shape[0],))
  for i, vec in enumerate(eigvec_nat.T):
    dot_prod = np.dot(intermediate_output_nat, np.conj(vec))
    Rs_nat[i] = np.linalg.norm(dot_prod)
  R_nat2 = 1 / N_features * np.sum(np.square(Rs_nat))

  # Calculate the radius of intermediate_output_adv along the eigenvectors
  Rs_adv = np.zeros((eigvec_adv.shape[0],))
  for i, vec in enumerate(eigvec_adv.T):
    dot_prod = np.dot(intermediate_output_adv, np.conj(vec))
    Rs_adv[i] = np.linalg.norm(dot_prod)
  R_adv2 = 1 / N_features * np.sum(np.square(Rs_adv))

  # terms to calculate SNR
  term_signal = np.linalg.norm(centroid_nat - centroid_adv)**2
  term_signal /= R_nat2
  print("term_signal: ", term_signal)

  term_bias = ( R_adv2/R_nat2 - 1 )
  m_examples = intermediate_output_nat.shape[0]
  term_bias /= m_examples
  print("term_bias: ", term_bias)

  term_dimension = ((R_nat2**2) / np.sum(Rs_nat**4)) ** (-1)
  term_dimension /= m_examples
  print("term_dimension: ", term_dimension)

  delta_centroids = centroid_nat - centroid_adv

  
  U_adv = (eigvec_adv * Rs_adv)/(R_adv2**0.5)
  term_SN_overlap_adv = np.linalg.norm(np.matmul(delta_centroids,U_adv))**2
  term_SN_overlap_adv /= m_examples 
  print("term_SN_overlap_adv: ", term_SN_overlap_adv)

  U_nat = (eigvec_nat * Rs_nat)/(R_nat2**0.5)
  term_SN_overlap_nat = np.linalg.norm(np.matmul(delta_centroids,U_nat))**2
  print("term_SN_overlap_nat: ", term_SN_overlap_nat)

  SNR_num = (term_signal + term_bias)
  SNR_den = (term_dimension + term_SN_overlap_adv + term_SN_overlap_nat) ** (0.5)

  return 0.5 * SNR_num / SNR_den

In [ ]:
df = pd.DataFrame(columns=["dataset","attack"])

data = [    
   {"dataset": "MNIST", "attack": "deepfool", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/MNIST-model_1-deepfool_eta_0_01_mi_20_ns_50.npz"},
   {"dataset": "MNIST", "attack": "FGSM", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/MNIST-model_1-FGSM_eps_0_062_ns_50.npz"},
   {"dataset": "MNIST", "attack": "CW2", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/MNIST-model_1-CW2_kap_0_1_cte_0_011_cteupd_0_011_mi_20_ns_50.npz"},

   {"dataset": "CIFAR10", "attack": "deepfool", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/CIFAR10-model_1-deepfool_eta_0_45_mi_20_ns_50.npz"},
   {"dataset": "CIFAR10", "attack": "FGSM", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/CIFAR10-model_1-FGSM_eps_0_005_ns_50.npz"},
   {"dataset": "CIFAR10", "attack": "CW2", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -7, "path": "../Codes/gen_data/CIFAR10-model_1-CW2_kap_0_1_cte_0_0032_cteupd_0_0032_mi_20_ns_50.npz"},

   {"dataset": "DR", "attack": "deepfool", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/DR-model_1-deepfool_eta_5_0_mi_20_ns_50.npz"},
   {"dataset": "DR", "attack": "FGSM", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/DR-model_1-FGSM_eps_0_001_ns_50.npz"},
   {"dataset": "DR", "attack": "CW2", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/DR-model_1-CW2_kap_0_1_cte_0_0005_cteupd_0_0005_mi_20_ns_50.npz"},

   {"dataset": "smallDR", "attack": "deepfool", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/smallDR-model_2-deepfool_eta_30_0_mi_20_ns_50.npz"},
   {"dataset": "smallDR", "attack": "FGSM", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/smallDR-model_2-FGSM_eps_0_001_ns_50.npz"},
   {"dataset": "smallDR", "attack": "CW2", "idx_noisy_layer_dense": -4, "idx_noisy_layer_conv": -11, "path": "../Codes/gen_data/smallDR-model_2-CW2_kap_0_1_cte_0_017_cteupd_0_017_mi_50_ns_50.npz"},
]

dfs = []
for d in data:
    dfs.append(pd.DataFrame(d, index=[0]))

df = pd.concat(dfs, ignore_index=True)

# add columns for SNR with nan values
df_columns_to_add = ["Dense SNR_nat", "Conv SNR_nat", "Dense SNR_adv", "Conv SNR_adv"]
for col in df_columns_to_add:
    df[col] = np.nan

df

In [ ]:
# fill table with SNR values
for idx, row in df.iterrows():
  # df.loc[idx, "Dense SNR_nat"] = compute_SNR(row["path"], row["idx_noisy_layer_dense"])
  # df.loc[idx, "Dense SNR_adv"] = compute_SNR(row["path"], row["idx_noisy_layer_dense"], adv=True)
  if idx != 9:
    continue
  df.loc[idx, "Conv SNR_nat"] = compute_SNR(row["path"], row["idx_noisy_layer_conv"])
  # df.loc[idx, "Conv SNR_adv"] = compute_SNR(row["path"], row["idx_noisy_layer_conv"], adv=True)

In [ ]:
df

In [ ]:
# Now manually, add a column "p_det" to df, which is the probability of detection for each attack on each dataset. So first add that column with zeros
df.loc[(df["dataset"] == "MNIST") & (df["attack"] == "deepfool"), "p_det_val"] = 0.99
df.loc[(df["dataset"] == "MNIST") & (df["attack"] == "FGSM"), "p_det_val"] = 0.80
df.loc[(df["dataset"] == "MNIST") & (df["attack"] == "CW2"), "p_det_val"] = 0.98

df.loc[(df["dataset"] == "CIFAR10") & (df["attack"] == "deepfool"), "p_det_val"] = 0.89
df.loc[(df["dataset"] == "CIFAR10") & (df["attack"] == "FGSM"), "p_det_val"] = 0.59
df.loc[(df["dataset"] == "CIFAR10") & (df["attack"] == "CW2"), "p_det_val"] = 0.87

df.loc[(df["dataset"] == "DR") & (df["attack"] == "deepfool"), "p_det_val"] = 0.54
df.loc[(df["dataset"] == "DR") & (df["attack"] == "FGSM"), "p_det_val"] = 0.43
df.loc[(df["dataset"] == "DR") & (df["attack"] == "CW2"), "p_det_val"] = 0.74

df.loc[(df["dataset"] == "smallDR") & (df["attack"] == "deepfool"), "p_det_val"] = 0.55
df.loc[(df["dataset"] == "smallDR") & (df["attack"] == "FGSM"), "p_det_val"] = 0.49
df.loc[(df["dataset"] == "smallDR") & (df["attack"] == "CW2"), "p_det_val"] = 0.55

df

In [ ]:
# save df to csv
df.to_csv("table.csv")

In [ ]:
from scipy.stats import linregress
axis_cols = 2
axis_rows = int(np.ceil(len(i_layers_to_test)/axis_cols))

fig, ax = plt.subplots(axis_rows, axis_cols, figsize=(15, 15),squeeze=False)
for i,i_layer in enumerate(i_layers_to_test[::-1]):
  ax_i = ax[i//axis_cols, i%axis_cols]
  ax_i.set_xlabel("SNR", fontsize=16)
  # ylabel using the latex syntax
  ax_i.set_ylabel(r"$p_{det}$", fontsize=16)
  ax_i.scatter(df["SNR layer " + str(i_layer)], df["p_det_val"])

  # Add a line for the best linear fit
  snrs = df["SNR layer " + str(i_layer)].to_numpy()
  p_dets = df["p_det_val"].to_numpy()

  # cast to float
  snrs = snrs.astype(float)
  p_dets = p_dets.astype(float)

  a, b = np.polyfit(snrs, p_dets, 1)
  r = np.corrcoef(snrs, p_dets)[0, 1]

  # Plot the line
  ax_i.plot(snrs, a*snrs + b, color='gray', label='y = {:.2f}x + {:.2f}'.format(a, b))
  ax_i.legend()

  # calculate best fit line
  slope, intercept, r_value, p_value, std_err = linregress(snrs, p_dets)

  ax_i.set_title(r"SNR layer index: " + str(i_layer) + " | $(R = {:.2f})$".format(r), fontsize=16)

plt.show()
path_history_save = "./graphs/SNRs.pdf"
fig.savefig(path_history_save, bbox_inches="tight", pad_inches=0)